# EatVul-Resources
EaTVul: ChatGPT-based Evasion Attack Against Software   Vulnerability Detection

This paper was accepted by USENIX Security '24. The source code and datasets provided are for research use only and not for commercial use.

## Idea
Machine learning-as-a-service has been widely applied in software security. However, Adversarial learning has long been a threat for cybersecurity. In the cybersecurity domain, especiaily within the topic of automatic vulnerability detection with deep learning techniques, there is lack of thorough assessment the security
issues when facing adversarial learning. In this paper, we explored the susceptibility of machine learning/deep neural vulnerability detector to adversarial attacks and developed an effective scheme to generate adversarial code and inject it into vulnerable samples to bypass deep neural detection systems.

Our proposed attackinv strategy consists of two main phases: adversarial data generation (①) and adversarial learning (②). And our work are based on the assumption that the adversary has no knowledge of the target model and cannot manipulate the training data. In the first stage, we train a surrogate model based on BiLSTM with an attention mechanism with knowleddge distaillation to approximate the target model. We identify significant non-vulnerable samples using SVM. Then, we retrieve the averaged attention scores from the attention layer to identify the key features that contribute significantly to the prediction. We further utilize chatGPT to generate adversarial data. In the second phase, we utilize a fuzzy genetic algorithm to select the best seed data, which is added to the vulnerable test case, aiming to bypass the machine learning-based software vulnerability detection system.

## About this repository

The "model" folder contains the Python code for:  
&emsp;&emsp;1. Loading the targte model -- ori_model.py & ori_model_run.py  
&emsp;&emsp;2. Train the surrogate model -- surrogate_model.py & surrogate_train.py  
&emsp;&emsp;3. Obtain the support vectors -- function get_support_vector_idx (in ori_model_run.py)  
&emsp;&emsp;4. Get the key tokens from the support vectors -- key_token_capture.py

To fine-tune the target model (in this case, CodeBERT works as an example.)via the following command:
```
python ori_model_run.py\
  --output_dir=./saved_newbap_models/model\
  --model_type=roberta \
  --tokenizer_name=microsoft/codebert-base \
  --model_name_or_path=microsoft/codebert-base \
  --do_train \
  --train_data_file=./example_train.json \
  --eval_data_file=./example_test.json \
  --test_data_file=./example_test.json \
  --epoch 10 \
  --block_size 400 \
  --train_batch_size 16 \
  --eval_batch_size 32 \
  --learning_rate 2e-5 \
  --max_grad_norm 1.0 \
  --evaluate_during_training \
  --seed 123456 2>&1 | tee train.log
```  
To train the surrogate model with knowledge distillation, please run the follwoing commands:
```
python surrogate_train.py
```
To evaluate the performance of the surrogate model, run the following commands:
```
python surrogate_test.py
```
The "data" folder conatins the data samples used in this paper to validate the findings, where "xxxxx_train.json" is the train dataset and "xxxxx_test.json" works as the test dataset. And the “xxxxx_test_ADV.json” is the testcase set with adversarial code snippets (with code snippet size of 1).  

The "code" folder contains two functional modules:  
&emsp;&emsp;1. Generate adversarial code snippets with chatGPT. -- adversarial_code_generation.py  
&emsp;&emsp;2. Use FGA to select the seed adversarial samples. -- fga_selection.py  






# Copy all files to /content

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r '/content/drive/MyDrive/EaTVul/content/files' '/content'
!pip install openai
!pip install transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/EaTVul/content/files': No such file or directory


In [2]:
import os
import shutil

source_directory = '/content/drive/MyDrive/files'
target_directory = '/content'

files_and_dirs = os.listdir(source_directory)

for entry in files_and_dirs:
    path = os.path.join(source_directory, entry)
    if os.path.isfile(path):
        shutil.copy2(path, target_directory)
    elif os.path.isdir(path):
        shutil.copytree(path, os.path.join(target_directory, os.path.basename(path)))

&emsp;&emsp;1. Loading the target model -- some sort of codeBERT     
&emsp;&emsp;2. Train the surrogate model -- surrogate_model.py & surrogate_train.py  
&emsp;&emsp;3. Obtain the support vectors -- function get_support_vector_idx (in ori_model_run.py) obtain them from non vulnerable samples   
&emsp;&emsp;4. Get the key tokens from the support vectors -- key_token_capture.py

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Sep  9 22:30:40 2018

@author: yuyu-
"""
import os
import pandas as pd
import csv
import pickle
import numpy as np
import datetime
import tensorflow as tf
import json

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing import text
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import TensorBoard, CSVLogger

from kd_train import *
from sur_model import BiLSTM_network

# from keras.layers import Input, Dense, Embedding, Bidirectional, LSTM, GlobalMaxPooling1D, merge, concatenate

MAX_LEN = 200
EMBEDDING_DIM = 100
EPOCHS = 100
BATCH_SIZE = 32
PATIENCE = 40

# 1. Load data.
def getData(file_path):
    """Extracts code snippets from a JSON file."""
    data = []
    with open(file_path, 'r') as f:
      for line in f:
        func = json.loads(line)
        data.append(func['func'])
    return data

def getID(file_path):
    """Extracts IDs and labels from a JSON file."""
    labels = []
    ids = []
    with open(file_path, 'r') as f:
      for line in f:
        label = json.loads(line)
        labels.append(label['target'])
        ids.append(label['idx'])
    return ids, labels

total_code_snippets = getData('asterisk_ast_train.json')
total_ids, total_labels = getID('asterisk_ast_train.json')

# Now you can separate the vulnerable and non-vulnerable snippets if needed
vulnerable_snippets = [code for code, label in zip(total_code_snippets, total_labels) if label == 1]
not_vulnerable_snippets = [code for code, label in zip(total_code_snippets, total_labels) if label == 0]

# Similarly for IDs
vulnerable_ids = [id for id, label in zip(total_ids, total_labels) if label == 1]
not_vulnerable_ids = [id for id, label in zip(total_ids, total_labels) if label == 0]

vulnerable_labels = [1] * len(vulnerable_snippets)
not_vulnerable_labels = [0] * len(not_vulnerable_snippets)

total_list = vulnerable_snippets + not_vulnerable_snippets
total_list_label = vulnerable_labels + not_vulnerable_labels
total_list_id = vulnerable_ids + not_vulnerable_ids

#--------------------------------------------------------#
# 2. Tokenization: convert the loaded text to tokens.
new_total_token_list = []

for sub_list_token in total_list:
    # Split the input string into a list of words
    words = sub_list_token.split()

    # Join the words with a comma
    new_line = ','.join(words)
    # new_line = ','.join(sub_list_token)
    new_total_token_list.append(new_line)

tokenizer = text.Tokenizer(num_words=None, filters=',', lower=False, char_level=False)
tokenizer.fit_on_texts(new_total_token_list)

# Save the tokenizer.
with open('binary_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=2)

# ----------------------------------------------------- #
# 3. Train a Vocabulary with Word2Vec -- using the function provided by gensim
w2vModel = Word2Vec(total_list, workers = 12)

print ("----------------------------------------")
print ("The trained word2vec model: ")
print (w2vModel)

w2vModel.wv.save_word2vec_format("binary_model_CBOW.txt", binary=False)

w2v_model_path = 'binary_model_CBOW.txt'
w2v_model = open(w2v_model_path)

# 4. Use the trained tokenizer to tokenize the sequence.
#-------------------------------------------------------------------
total_sequences = tokenizer.texts_to_sequences(new_total_token_list)
word_index = tokenizer.word_index
print ('Found %s unique tokens.' % len(word_index))

print ("The length of tokenized sequence: " + str(len(total_sequences)))

#------------------------------------#
# 5. Do the paddings.
#--------------------------------------------------------
print ("max_len ", MAX_LEN)
print('Pad sequences (samples x time)')
total_sequences_pad = pad_sequences(total_sequences, maxlen = MAX_LEN, padding ='post')
print ("The shape after paddings: ")
print (total_sequences_pad.shape)

train_set_x, validation_set_x, train_set_y, validation_set_y, train_set_id, validation_set_id = train_test_split(total_sequences_pad, total_list_label, total_list_id, test_size=0.3, random_state=42)
test_set_x, validation_set_x, test_set_y, validation_set_y, test_set_id, validation_set_id = train_test_split(validation_set_x, validation_set_y, validation_set_id, test_size=0.5, random_state=42)

print ("Training set: ")
print(train_set_x.shape)
print ("Validation set: ")
print (validation_set_x.shape)
print ("Test set: ")
print(test_set_x.shape)

# Save the test data sets.
#--------------------------------------------------------------
with open('test_set_x.pickle', 'wb') as handle:
    pickle.dump(test_set_x, handle, protocol=2)

with open('test_set_y.pickle', 'wb') as handle:
    pickle.dump(test_set_y, handle, protocol=2)

train_set_y = np.asarray(train_set_y)
validation_set_y = np.asarray(validation_set_y)
test_set_y = np.asarray(test_set_y)

print (len(train_set_x), len(train_set_y), len(train_set_id), len(validation_set_x),  len(validation_set_y), len(validation_set_id))

print (train_set_x.shape, train_set_y.shape, validation_set_x.shape, validation_set_y.shape, test_set_x.shape, test_set_y.shape, )

print (np.count_nonzero(train_set_y), np.count_nonzero(validation_set_y), np.count_nonzero(test_set_y))

# -----------------------------------
# 6. Preparing the Embedding layer

embeddings_index = {}

for line in w2v_model:
   end_idx = 0
   if not line.isspace():
       values = line.split()
       for i in range(len(values)):
           if values[i].startswith('0.') or values[i].startswith('-0.'):
              end_idx = i
              break
       word = values[ : end_idx]
       word = "".join(word)
       coefs = np.asarray(values[end_idx:], dtype='float32')
       embeddings_index[word] = coefs
w2v_model.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
   embedding_vector = embeddings_index.get(word)
   if embedding_vector is not None:
       # words not found in embedding index will be all-zeros.
       embedding_matrix[i] = embedding_vector

----------------------------------------
The trained word2vec model: 
Word2Vec<vocab=94, vector_size=100, alpha=0.025>
Found 10999 unique tokens.
The length of tokenized sequence: 880
max_len  200
Pad sequences (samples x time)
The shape after paddings: 
(880, 200)
Training set: 
(616, 200)
Validation set: 
(132, 200)
Test set: 
(132, 200)
616 616 616 132 132 132
(616, 200) (616,) (132, 200) (132,) (132, 200) (132,)
50 10 10
Found 94 word vectors.


In [4]:
from keras.utils import to_categorical

# Update this in your training function before calling model.fit()
train_set_y = to_categorical(train_set_y, num_classes=2)  # Change to 2 classes
validation_set_y = to_categorical(validation_set_y, num_classes=2)
test_set_y = to_categorical(test_set_y, num_classes=2)


In [5]:
# -----------------------------------
# 7. The training of the BiLSTM surrogate model

LOSS_FUNCTION = 'binary_crossentropy'
OPTIMIZER = 'adamax'

def normal_train(train_set_x, train_set_y, validation_set_x, validation_set_y, saved_model_name):
    model = BiLSTM_network(MAX_LEN, EMBEDDING_DIM, word_index, embedding_matrix, True)
    model.compile(loss=LOSS_FUNCTION,
             optimizer=OPTIMIZER,
             metrics=[tf.keras.metrics.F1Score()])
    callbacks_list = [
       ModelCheckpoint(filepath = model_saved_path + os.sep + saved_model_name +'_{epoch:02d}_{val_loss:3f}.keras', monitor='val_loss', verbose=2, save_best_only=True),
       EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=2, mode="min"),
		# TensorBoard(log_dir=log_path,  write_graph=True, write_images=True, embeddings_freq=0, embeddings_metadata=None),
       CSVLogger(log_path + os.sep + saved_model_name + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.log')]

    model.fit(train_set_x, train_set_y,
         epochs=EPOCHS,
         batch_size=BATCH_SIZE,
		   shuffle = False, # The data has already been shuffle before, so it is unnessary to shuffle it again. (And also, we need to correspond the ids to the features of the samples.)
         #validation_split=0.5,
         validation_data = (validation_set_x, validation_set_y), # Validation data is not used for training (or development of the model)
         callbacks=callbacks_list, # Get the best weights of the model and stop the first round training.
         verbose=2)
    model.summary()

    return model

model_saved_path = 'models'
log_path = 'logs'
if not os.path.exists(log_path):
    os.makedirs(log_path)
# dense_3 = Dense(2, activation='softmax')(dense_2)
model = normal_train(train_set_x, train_set_y, validation_set_x, validation_set_y, 'BiLSTM_binary_attention')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<KerasTensor shape=(None, 256, 200), dtype=float32, sparse=False, name=keras_tensor_5>
<KerasTensor shape=(None, 256, 200), dtype=float32, sparse=False, name=keras_tensor_6>
<KerasTensor shape=(None, 256, 200), dtype=float32, sparse=False, name=keras_tensor_7>
a_probs is
<KerasTensor shape=(None, 200, 256), dtype=float32, sparse=False, name=keras_tensor_8>
<KerasTensor shape=(None, 200, 512), dtype=float32, sparse=False, name=keras_tensor_9>
Epoch 1/100

Epoch 1: val_loss improved from inf to 0.22531, saving model to models/BiLSTM_binary_attention_01_0.225311.keras
20/20 - 12s - 585ms/step - f1_score: 0.4985 - loss: 0.3224 - val_f1_score: 0.4803 - val_loss: 0.2253
Epoch 2/100

Epoch 2: val_loss did not improve from 0.22531
20/20 - 1s - 39ms/step - f1_score: 0.5170 - loss: 0.2293 - val_f1_score: 0.4803 - val_loss: 0.2320
Epoch 3/100

Epoch 3: val_loss did not improve from 0.22531
20/20 - 1s - 63ms/step - f1_score: 0.5181 - loss: 0.2047 - val_f1_score: 0.4803 - val_loss: 0.2433
Epoch 4/1

Model: "BiLSTM_network"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 200)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 200, 100)       │      1,100,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 200, 128)       │         84,480 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 200, 128)       │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 200, 256)       │        263,168 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute (Permute)         │ (None, 256, 200)       │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 256, 200)       │              0 │ permute[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256, 200)       │         40,200 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_vec (Permute)   │ (None, 200, 256)       │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 200, 512)       │              0 │ bidirectional_1[0][0], │
│                           │                        │                │ attention_vec[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 102400)         │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 102400)         │              0 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │      6,553,664 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 32)             │          2,080 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 2)              │             66 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,930,976 (83.66 MB)

 Trainable params: 6,943,658 (26.49 MB)

 Non-trainable params: 1,100,000 (4.20 MB)

 Optimizer params: 13,887,318 (52.98 MB)

In [6]:
from sklearn.svm import SVC

# added by shigang  last second layer feature obtain
def ObtainRepresentations(input_sequences, layer_number, model):
    layered_model = Model(inputs = model.input, outputs=model.layers[layer_number].output)
    representations = layered_model.predict(input_sequences)
    return representations

def get_support_vector_idx(sample_vec, sample_label):
    svclassifier = SVC(kernel='rbf')  # Initialize SVM classifier with a radial basis function kernel
    svclassifier.fit(sample_vec, sample_label)  # Train the SVM on the provided sample vectors and their corresponding labels
    sv_idx = svclassifier.support_  # Retrieve indices of the support vectors

    return sv_idx  # Return the indices of the support vectors

In [7]:
layer_number = 1
input_sequences = train_set_x

# Call the ObtainRepresentations function
feature_representations = ObtainRepresentations(input_sequences, layer_number, model)
print("Feature Representations Shape:", feature_representations.shape)

# Flatten the feature representations
flattened_representations = feature_representations.reshape(feature_representations.shape[0], -1)

sv_idx = get_support_vector_idx(flattened_representations, train_set_y[:,0])
print("SVM indices:", sv_idx)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Feature Representations Shape: (616, 200, 100)
SVM indices: [  4  10  18  34  38  42  46  55  59  64  99 101 102 120 121 133 135 160
 174 204 213 225 233 250 255 265 283 288 290 299 305 334 341 343 450 470
 483 484 487 499 505 506 508 514 538 543 555 578 583 608   0   2   5   6
   7   8   9  11  13  16  17  20  21  24  25  26  27  28  29  30  31  33
  35  36  39  43  45  48  49  50  52  53  54  56  57  58  60  61  62  66
  67  68  69  70  73  74  75  76  77  78  79  80  81  82  84  86  87  88
  91  92  93  94  95  97  98 100 103 104 106 107 109 110 112 113 116 117
 119 122 123 124 126 127 128 129 130 134 136 137 138 139 140 142 147 149
 150 151 153 155 156 157 159 161 162 163 164 166 167 168 169 170 172 173
 175 179 180 181 183 185 186 187 190 191 193 194 195 196 197 202 203 205
 207 209 210 211 212 214 215 216 219 220 221 223 224 226 227 228 230 231
 232 234 235 237 238 239 240 241 244 247 248 249 251 253 254 256 257 259
 260 264 270 271 272 275 

In [8]:
import numpy as np
import pandas as pd
from keras.models import Model
import tensorflow as tf

def ListToCSV(list_to_csv, path):
    df = pd.DataFrame(list_to_csv)
    df.to_csv(path, index=False)

def visualize_attention(test_seq, i, model, id2wrd, n):
    """
    Visualize the top n words the model pays attention to in the i-th sequence.
    """
    print(id2wrd)
    # Invert the word index
    id2wrd = dict((v, k) for k, v in id2wrd.items())
    print(id2wrd)

    # Create a new model that outputs both the required layer output and the attention weights
    intermediate_layer_output = Model(inputs=model.input, outputs=[model.layers[4].output, model.layers[5].output])

    # Then retrieve the outputs
    outputs, attention_weights = intermediate_layer_output.predict(test_seq)

    for i in range(len(outputs)):
        eij = np.tanh(np.dot(outputs[i], attention_weights[0]))
        ai = np.exp(eij)
        weights = ai / np.sum(ai, axis=-1, keepdims=True)
        weights_summed = np.sum(weights, axis=1)

        # Handle top n keys
        top_keys = np.argpartition(weights_summed, -n)[-n:]

        # Assuming ListToCSV is a custom function you've defined to write lists to CSV
        ListToCSV(eij, 'eij.csv')
        ListToCSV(ai, 'ai.csv')
        ListToCSV(weights, '1weights.csv')
        ListToCSV(weights_summed, '2weights.csv')

    # Print the sequence
    print(' '.join([id2wrd[wrd_id] for wrd_id in test_seq[i] if wrd_id != 0]))
    attention_words = []
    print('--------------------------Attentive Words start: --------------------------------------')
    for k in test_seq[i][top_keys]:
        if k != 0:
            attention_words.append(id2wrd[k])
            print(id2wrd[k])
    print('--------------------------Attentive Words end: --------------------------------------')
    return attention_words

def printAttentionWords(test_set_x, test_set_y, model, word_index, num_word_pay_atten):
    vul_index = [index for index, item in enumerate(test_set_y) if item == 1]
    return visualize_attention(test_set_x, item, model, word_index, num_word_pay_atten)

In [9]:
# -*- coding: utf-8 -*-
from random import randrange
import os
import csv
import pickle
import numpy as np
import pandas as pd

from keras.layers import Input
from keras.models import load_model, Model

from keras.preprocessing.sequence import pad_sequences
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import datetime
import keras.backend as K

LOSS_FUNCTION = 'binary_crossentropy'
OPTIMIZER = 'adamax'

MAX_LEN = 200
EMBEDDING_DIM = 100
BATCH_SIZE = 64

print ("The model has been loaded: ")
print (model.summary())

def LoadSavedData(path):
    with open(path, 'rb') as f:
        loaded_data = pickle.load(f)
    return loaded_data

def ListToCSV(list_to_csv, path):
    df = pd.DataFrame(list_to_csv)
    df.to_csv(path, index=False)

test_set_x = LoadSavedData('test_set_x.pickle')
test_set_y = LoadSavedData('test_set_y.pickle')

print(test_set_x.shape)
print(len(test_set_y))

tokenizer = LoadSavedData('binary_tokenizer.pickle')
word_index = tokenizer.word_index
print ("max_len ", MAX_LEN)
print('Pad sequences (samples x time)')

probs = model.predict(test_set_x, batch_size = BATCH_SIZE, verbose=1)

with open('probs.pickle', 'wb') as handle:
    pickle.dump(probs, handle, protocol=2)

ListToCSV(probs.tolist(), 'prob_sur_model.csv')

predicted_classes = []

for item in probs:
    if item[0] > 0.5:
        predicted_classes.append(1)
    else:
        predicted_classes.append(0)

ListToCSV(predicted_classes, 'classes_assembly.csv')
test_accuracy = np.mean(np.equal(test_set_y, predicted_classes))
test_set_y = np.asarray(test_set_y)

print ("Surrogate Model classification result: ")
target_names = ["Non-vulnerable","Vulnerable"] #non-vulnerable->0, vulnerable->1
print (confusion_matrix(test_set_y, predicted_classes, labels=[0,1]))
print ("\r\n")
print ("\r\n")
print (classification_report(test_set_y, predicted_classes, target_names=target_names))

attention_words = printAttentionWords(test_set_x, test_set_y, model, word_index, 20)

The model has been loaded: 


Model: "BiLSTM_network"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 200)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 200, 100)       │      1,100,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 200, 128)       │         84,480 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 200, 128)       │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 200, 256)       │        263,168 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute (Permute)         │ (None, 256, 200)       │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 256, 200)       │              0 │ permute[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256, 200)       │         40,200 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_vec (Permute)   │ (None, 200, 256)       │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 200, 512)       │              0 │ bidirectional_1[0][0], │
│                           │                        │                │ attention_vec[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 102400)         │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 102400)         │              0 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │      6,553,664 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 32)             │          2,080 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 2)              │             66 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,930,976 (83.66 MB)

 Trainable params: 6,943,658 (26.49 MB)

 Non-trainable params: 1,100,000 (4.20 MB)

 Optimizer params: 13,887,318 (52.98 MB)

None
(132, 200)
132
max_len  200
Pad sequences (samples x time)
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 216ms/step
Surrogate Model classification result: 
[[  7 115]
 [  0  10]]




                precision    recall  f1-score   support

Non-vulnerable       1.00      0.06      0.11       122
    Vulnerable       0.08      1.00      0.15        10

      accuracy                           0.13       132
     macro avg       0.54      0.53      0.13       132
  weighted avg       0.93      0.13      0.11       132

{'LEAF_NODE': 1, ';': 2, '5': 3, '(': 4, ')': 5, 'int': 6, 'struct': 7, '*': 8, '6': 9, '8': 10, '7': 11, '9': 12, '=': 13, '4': 14, '1': 15, 'char': 16, '[': 17, ']': 18, 'unsigned': 19, 'SIMPLE_DECL': 20, 'NAME': 21, 'TYPE_NAME': 22, 'TYPE': 23, '10': 24, 'FIELD': 25, 'VAR_DECL': 26, '3': 27, 'enum': 28, '<<': 29, '{': 30, '}': 31, '2': 32, '->': 33, 'BASE_CLASSES': 34, '11': 35, 'AST_STRING_FIELD': 36, '12': 37, ':': 38, 'ARGUMENT': 39, 'chan': 40, '13': 41, 'EXPR': 42, '0': 43, '14'

4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step 

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step
FUNCTION_DEF 1 RETURN_TYPE 2 static int LEAF_NODE 3 static TYPE_NAME 3 int LEAF_NODE 4 int FUNCTION_NAME 2 stub_resolve LEAF_NODE 3 stub_resolve PARAMETER_LIST 2 ( struct ast_dns_query * query ) LABEL 3 e : LEAF_NODE 4 e LEAF_NODE 4 : EXPR_STATEMENT 3 print ( "s" ) ; EXPR 4 print ( "s" ) FUNCTION_CALL 5 print ( "s" ) CALLEE 6 print LEAF_NODE 7 print LEAF_NODE 6 ( ARGUMENT 6 "s" FIELD 7 "s" LEAF_NODE 8 "s" LEAF_NODE 6 ) LEAF_NODE 4 ; LEAF_NODE 3 ( PARAMETER_DECL 3 struct ast_dns_query * query TYPE 4 struct ast_dns_query * TYPE_NAME 5 struct ast_dns_query LEAF_NODE 6 struct LEAF_NODE 6 ast_dns_query POINTER 5 * LEAF_NODE 6 * NAME 4 query LEAF_NODE 5 query LEAF_NODE 3 ) LEAF_NODE 2 { STATEMENTS 2 JUMP_STATEMENT 3 return 0 ; KEYWORD 4 return LEAF_NODE 5 return DESTINATION 4 0 EXPR 5 0 FIELD 6 0 LEAF_NODE 7 0 LEAF_NODE 4 ; LEAF_NODE 2 }
--------------------------Attentive Words start: --------------------------------------
6
LEAF_NODE
print
LEAF_NODE
F

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name = 'gpt2-large'  # or 'gpt2-medium', 'gpt2-large', etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model_gpt = GPT2LMHeadModel.from_pretrained(model_name)

# Set the model to evaluation mode
model_gpt.eval()

def generate_adv_snippet(prompt):
    # Tokenize input prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # Generate a response
    with torch.no_grad():
        outputs = model_gpt.generate(
            inputs,
            max_length=1000,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True,
            temperature=0.7
        )

    # Decode the response
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return answer

def generate_new_prompt(content, pre_context, post_context, attention_words, source_code):
    new_prompts = []
    for idx in range(len(content)):
        prompt = content[idx]['prompt'].replace('\n', '').replace('\r', '')
        pre_con = pre_context[idx]
        post_con = post_context[idx]

        # Incorporate attention words and source code into the prompt
        attention_str = "Focus on the following important words: " + ", ".join(attention_words[idx]) + "."

        front_context = "With the partial preceding codes provided as:"
        end_context = "and with the partial following codes provided as"

        # Construct the new prompt with attention words and source code
        new_prompt = f"{front_context} {pre_con}. {attention_str} Given the source code: {source_code} {prompt} {end_context} {post_con}"
        new_prompts.append(new_prompt)
    return new_prompts

def save_adv_snippet(new_prompts):
    adv_snippets = []
    for new_prompt in new_prompts:
        answer = generate_adv_snippet(new_prompt)
        print("answer", answer)
        adv_snippets.append(answer)
    return adv_snippets

# Example content, pre_context, post_context, attention words, and source code
content = [{'prompt': "Generate an adversarial example based on the provided C code.\n"}]
pre_context = ['def generate_adversarial_example():']
post_context = ['# End of function']

# Provided source code
source_code = total_code_snippets[0]

# Construct the new prompts
new_prompts = generate_new_prompt(content, pre_context, post_context, attention_words, source_code)

# Generate adversarial snippets
adv_snippets = save_adv_snippet(new_prompts)

# Print the generated adversarial snippets
for snippet in adv_snippets:
    print(snippet)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


answer With the partial preceding codes provided as: def generate_adversarial_example():. Focus on the following important words: 6. Given the source code: FUNCTION_DEF 1 RETURN_TYPE 2 static int LEAF_NODE 3 static TYPE_NAME 3 int LEAF_NODE 4 int FUNCTION_NAME 2 g729_length LEAF_NODE 3 g729_length PARAMETER_LIST 2 ( unsigned int samples ) LEAF_NODE 3 ( PARAMETER_DECL 3 unsigned int samples TYPE 4 unsigned int TYPE_NAME 5 unsigned int LEAF_NODE 6 unsigned LEAF_NODE 6 int NAME 4 samples LEAF_NODE 5 samples LEAF_NODE 3 ) LEAF_NODE 2 { STATEMENTS 2 JUMP_STATEMENT 3 return samples / 8 ; KEYWORD 4 return LEAF_NODE 5 return DESTINATION 4 samples / 8 EXPR 5 samples / 8 FIELD 6 samples LEAF_NODE 7 samples LEAF_NODE 6 / FIELD 6 8 LEAF_NODE 7 8 LEAF_NODE 4 ; LEAF_NODE 2 } FUNCTION_DEF 1 RETURN_TYPE 2 int TYPE_NAME 3 int LEAF_NODE 4 int FUNCTION_NAME 2 main LEAF_NODE 3 main PARAMETER_LIST 2 ( ) LABEL 3 e : LEAF_NODE 4 e LEAF_NODE 4 : EXPR_STATEMENT 3 print ( "s" ) ; EXPR 4 print ( "s" ) FUNCTION_C

In [16]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model and tokenizer
model_name = 'gpt2-large'  # You can use 'gpt2', 'gpt2-medium', or 'gpt2-large'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model_gpt = GPT2LMHeadModel.from_pretrained(model_name)

# Set the model to evaluation mode
model_gpt.eval()

def generate_adv_snippet(prompt):
    # Tokenize input prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # Generate a response
    with torch.no_grad():
        outputs = model_gpt.generate(
            inputs,
            max_length=1000,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True,
            temperature=0.7
        )

    # Decode the response
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return answer

def generate_new_prompt(content, pre_context, post_context, attention_words, source_code):
    new_prompts = []
    for idx in range(len(content)):
        prompt = content[idx]['prompt'].replace('\n', '').replace('\r', '')
        pre_con = pre_context[idx]
        post_con = post_context[idx]

        # Incorporate attention words and source code into the prompt
        attention_str = "Focus on the following important words: " + attention_words + "."

        front_context = "With the partial preceding codes provided as:"
        end_context = "and with the partial following codes provided as"

        # Construct the new prompt with attention words and source code
        new_prompt = f"{front_context} {pre_con}. {attention_str} Given the source code: {source_code} {prompt} {end_context} {post_con}"
        new_prompts.append(new_prompt)
    return new_prompts

def save_adv_snippet(new_prompts):
    adv_snippets = []
    for new_prompt in new_prompts:
        answer = generate_adv_snippet(new_prompt)
        print("Generated adversarial example:", answer)
        adv_snippets.append(answer)
    return adv_snippets

# Example content, pre_context, post_context, attention words, and source code
content = [{'prompt': "Generate an adversarial example based on the provided C code.\n"}]
pre_context = ['def generate_adversarial_example():']
post_context = ['# End of function']

# Example attention words and source code
source_code = total_code_snippets[0]

# Construct the new prompts
new_prompts = generate_new_prompt(content, pre_context, post_context, attention_words, source_code)

# Generate adversarial snippets
adv_snippets = save_adv_snippet(new_prompts)

# Print the generated adversarial snippets
for snippet in adv_snippets:
    print("Adversarial Snippet:\n", snippet)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated adversarial example: With the partial preceding codes provided as: def generate_adversarial_example():. Focus on the following important words: 6. Given the source code: FUNCTION_DEF 1 RETURN_TYPE 2 static int LEAF_NODE 3 static TYPE_NAME 3 int LEAF_NODE 4 int FUNCTION_NAME 2 g729_length LEAF_NODE 3 g729_length PARAMETER_LIST 2 ( unsigned int samples ) LEAF_NODE 3 ( PARAMETER_DECL 3 unsigned int samples TYPE 4 unsigned int TYPE_NAME 5 unsigned int LEAF_NODE 6 unsigned LEAF_NODE 6 int NAME 4 samples LEAF_NODE 5 samples LEAF_NODE 3 ) LEAF_NODE 2 { STATEMENTS 2 JUMP_STATEMENT 3 return samples / 8 ; KEYWORD 4 return LEAF_NODE 5 return DESTINATION 4 samples / 8 EXPR 5 samples / 8 FIELD 6 samples LEAF_NODE 7 samples LEAF_NODE 6 / FIELD 6 8 LEAF_NODE 7 8 LEAF_NODE 4 ; LEAF_NODE 2 } FUNCTION_DEF 1 RETURN_TYPE 2 int TYPE_NAME 3 int LEAF_NODE 4 int FUNCTION_NAME 2 main LEAF_NODE 3 main PARAMETER_LIST 2 ( ) LABEL 3 e : LEAF_NODE 4 e LEAF_NODE 4 : EXPR_STATEMENT 3 print ( "s" ) ; EXPR 4 

In [17]:
print(attention_words)

['6', 'LEAF_NODE', 'print', 'LEAF_NODE', 'FIELD', '4', '7', '"s"', 'LEAF_NODE', 'LEAF_NODE', '8', '"s"', '4', 'LEAF_NODE', '6', '2', ')', '{']
